In [6]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pathlib import Path

In [ ]:
def create_emptydf(start_date,end_date):
    """
    Creates empty DataFrame with date range
    Args:
        start_date (str): Start date in 'yyyy-mm-dd' format
        end_date (str): End date in 'yyyy-mm-dd' format
        
    Returns:
        empty (df): Eempty df ready for population
    """
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    date_range = pd.date_range(start, end)

    df = pd.DataFrame({'Date': date_range})
    
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df['nr.sessions'] = 0
    df['total km'] = 0.0
    df['km Z3-4'] = 0.0
    df['km Z5-T1-T2'] = 0.0
    df['hours alternative'] = 0.0
    return df

In [5]:
empty = create_emptydf('2024-08-01','2024-09-30')
empty.head(10)

,Date,nr.sessions,total km,km Z3-4,km Z5-T1-T2,hours alternative
0,2024-08-01,0,0.0,0.0,0.0,0.0
1,2024-08-02,0,0.0,0.0,0.0,0.0
2,2024-08-03,0,0.0,0.0,0.0,0.0
3,2024-08-04,0,0.0,0.0,0.0,0.0
4,2024-08-05,0,0.0,0.0,0.0,0.0
5,2024-08-06,0,0.0,0.0,0.0,0.0
6,2024-08-07,0,0.0,0.0,0.0,0.0
7,2024-08-08,0,0.0,0.0,0.0,0.0
8,2024-08-09,0,0.0,0.0,0.0,0.0
9,2024-08-10,0,0.0,0.0,0.0,0.0


In [22]:
def readfiles(file_path="../data/external"):
    '''
    Creates lists of all csv files in directory

    Args: 
        file_path (str): the relative path for the folder that 
        contains all the activity files

    Returns:
        run_activities (list):
    '''
    fpath = Path(file_path)

    run_activities = list(fpath.glob(f'*Running_*.csv'))

    other_activities = list(fpath.glob(f'*[!Running]_*.csv'))

    return run_activities,other_activities

In [23]:
def readrun(file):
    df = pd.read_csv(file)
    return df

In [27]:
runs,other =readfiles()
print(len(runs))
for file in runs:

    df_temp = readrun(file)
    total_km=df_temp['Distance'].iloc[-1]
    print(total_km)

30
6.38
4.39
0.8
1.54
6.11
6.18
9.84
12.53
6.43
9.84
6.3
5.27
3.25
2.61
10.1
7.29
2.37
10.12
12
10.02
2.43
11.78
9.32
7.41
15.27
5.01
3.82
11.02
10.6
4.68
